In [1]:
# import libraries
import pandas as pd
import numpy as np
# from IPython.display import display, HTML

# read Event_Codes
Event_Codes = pd.read_csv("Event_Codes.txt", sep = '\t')

# read Game_Lineup
Game_Lineup = pd.read_csv("Game_Lineup.txt", sep = '\t')

# read Play_by_Play
Play_by_Play = pd.read_csv("Play_by_Play.txt", sep = '\t')

In [2]:
# transform Event_Codes

# add description variable
Event_Codes.Event_Msg_Type_Description = [x.strip(' ') for x in Event_Codes.Event_Msg_Type_Description]
Event_Codes['Event_Description'] = Event_Codes.Event_Msg_Type_Description + " | " + Event_Codes.Action_Type_Description

Event_Codes.head()

,Event_Msg_Type,Action_Type,Event_Msg_Type_Description,Action_Type_Description,Event_Description
0,1,0,Made Shot,No Shot,Made Shot | No Shot
1,1,1,Made Shot,Jump Shot,Made Shot | Jump Shot
2,1,2,Made Shot,Running Jump Shot,Made Shot | Running Jump Shot
3,1,3,Made Shot,Hook Shot,Made Shot | Hook Shot
4,1,4,Made Shot,Tip Shot,Made Shot | Tip Shot


In [3]:
# transform Play_by_Play

# drop unimportant columns
Play_by_Play = Play_by_Play.drop(['Option2', 'Option3', 'Team_id', 'Team_id_type', 'Person1_type', 'Person2_type', 'Person3_type'], axis=1)

# sort by Game_id, Period, PC_Time, WC_Time, and Event_Num
Play_by_Play = Play_by_Play.sort_values(['Game_id', 'Period', 'PC_Time', 'WC_Time', 'Event_Num'], ascending=[True, True, False, True, True])

Play_by_Play.head()

,Game_id,Event_Num,Event_Msg_Type,Period,WC_Time,PC_Time,Action_Type,Option1,Person1,Person2,Person3
0,006728e4c10e957011e1f24878e6054a,2,12,1,546410,7200,0,0,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e
1,006728e4c10e957011e1f24878e6054a,4,10,1,546420,7200,0,0,8d2127290c94bd41b82a2938734bc750,99104de2626f67c1fa2ce70504970c3f,766802a8fda500d7945950de7398c9c6
2,006728e4c10e957011e1f24878e6054a,7,1,1,546590,7020,79,2,42e0d7167f04a4ff958c6442da0e6851,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e
3,006728e4c10e957011e1f24878e6054a,8,6,1,546620,7020,2,0,616281dee946056b071699476fdee9ec,42e0d7167f04a4ff958c6442da0e6851,0b1c631effab25d4af2d16fc123c3d31
4,006728e4c10e957011e1f24878e6054a,10,3,1,546810,7020,10,1,42e0d7167f04a4ff958c6442da0e6851,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e


In [4]:
# merge Play_by_Play and Event_Codes
df = pd.merge(Play_by_Play, Event_Codes[['Event_Msg_Type', 'Action_Type', 'Event_Description']], how='left', on=['Event_Msg_Type', 'Action_Type'])

# merge with Game_Lineup to find Team ID
df = pd.merge(df, Game_Lineup[['Person_id', 'Team_id']][Game_Lineup.Period == 0].drop_duplicates(), how='left', left_on='Person1', right_on='Person_id', sort=False)
df.rename(columns={'Team_id':'Team1_id'}, inplace=True)
df = pd.merge(df, Game_Lineup[['Person_id', 'Team_id']][Game_Lineup.Period == 0].drop_duplicates(), how='left', left_on='Person2', right_on='Person_id')
df.rename(columns={'Team_id':'Team2_id'}, inplace=True)

# rearrange columns
df = df[['Game_id', 'Period', 'PC_Time', 'WC_Time', 'Event_Num', 'Event_Msg_Type', 'Action_Type', 'Event_Description', 'Option1', 'Person1', 'Person2', 'Team1_id', 'Team2_id']]

In [5]:
def does_row_end_possession(row):
    # possession is ended by:
    # (1) made field goal attempts
    if (row.Event_Msg_Type == 1):
        # return False if was also fouled
        if (df.iloc[row.name + 1].Event_Msg_Type == 6) & (row.PC_Time == df.iloc[row.name + 1].PC_Time):
            return False
        else:
            return True
    
    # (2) made final free throw attempt
    elif (row.Event_Msg_Type == 3 and row.Option1 == 1 and row.Action_Type in [10, 12, 15, 16, 17, 19, 20, 22, 26, 29]):
        return True
    
    # (3) missed final free throw attempt that results in a defensive rebound
    elif (row.Event_Msg_Type == 2 and row.Team1_id != df.iloc[row.name + 1].Team1_id):
        return True
    
    # (4) missed field goal attempt that results in a defensive rebound
    elif (row.Event_Msg_Type == 3 and row.Option1 != 1 and row.Action_Type in [10, 12, 15, 16, 17, 19, 20, 22, 26, 29] and row.Team1_id != df.iloc[row.name + 1].Team1_id):
        return True
    
    # (5) turnover
    elif (row.Event_Msg_Type == 5):
        return True
    
    # (6) end of time period.
    elif (row.Event_Msg_Type == 13):
        return True
    
    else:
        return False

In [6]:
df['End_Of_Possession'] = df.apply(does_row_end_possession, axis=1)
df['points_scored'] = df.loc[((df.Event_Msg_Type == 1) | ((df.Event_Msg_Type == 3) & (df.Option1 == 1))), 'Option1']

In [7]:
def allocate_points(row, t1, t2, t1_pts, t2_pts):
    if (row.in_game == 1):
        if (row.Team_id == t1):
            off_pts = row.off_pts + t1_pts
            def_pts = row.def_pts + t2_pts
        elif (row.Team_id == t2):
            off_pts = row.off_pts + t2_pts
            def_pts = row.def_pts + t1_pts
        return off_pts, def_pts
    else:
        return row.off_pts, row.def_pts

In [8]:
def allocate_possessions(row, t, t1, t2):
    if (row.in_game == 1):
        if (t in [t1, t2]):
            if (row.Team_id == t):
                return row.off_poss + 1.0, row.def_poss
            else:
                return row.off_poss, row.def_poss + 1.0
    else:
        return row.off_poss, row.def_poss

In [9]:
def substitute(row):
    players.loc[players.Person_id==row.Person1, 'in_game'] = 0
    players.loc[players.Person_id==row.Person2, 'in_game'] = 1

In [10]:
# list of game results
players_list = []

In [11]:
# for every game
for game in pd.unique(df.Game_id):
    
    # find all players
    players = Game_Lineup[(Game_Lineup.Game_id == game) & (Game_Lineup.Period == 0)].sort_values(by='Team_id')
    players['off_pts'] = 0
    players['def_pts'] = 0
    players['off_poss'] = 0
    players['def_poss'] = 0
    
    # find all teams
    team1, team2 = pd.unique(players.Team_id)
    
    # for every period
    for period in pd.unique(df.Period[(df.Game_id == game)]):
        
        # subset that game-period
        game_period = df[(df.Game_id == game) & (df.Period == period)]
        
        # find end of possession list
        end_of_possession_list = np.flatnonzero(game_period.End_Of_Possession)
        
        # find players who started
        players['in_game'] = 1*players.Person_id.isin(Game_Lineup[(Game_Lineup.Period == period) & (Game_Lineup.Game_id == game)].Person_id)
        
        # section game-period by possessions
        for i in range(0, len(end_of_possession_list)):
            
            # find possession
            if (i > 0): first_event_index = end_of_possession_list[i - 1] + 1; last_event_index = end_of_possession_list[i] + 1;
            else: first_event_index = 0; last_event_index = end_of_possession_list[0] + 1;
            possession = game_period[first_event_index:last_event_index]
          
            # allocate points to players in game
            team1_pts = np.nansum(possession.points_scored[(possession.Team1_id == team1) & (possession.points_scored > 0)])
            team2_pts = np.nansum(possession.points_scored[(possession.Team1_id == team2) & (possession.points_scored > 0)])
            players['off_pts'], players['def_pts'] = zip(*players.apply(allocate_points, t1=team1, t2=team2, t1_pts=team1_pts, t2_pts=team2_pts, axis=1))
            
            # assign offensive team
            off_team = possession.Team1_id.iat[-1]
            # if offensive team is assigned incorrectly, base it on the previous offensive team
            if off_team not in [team1, team2]:
                if previous_off_team == team1:
                    off_team = team2
                elif previous_off_team == team2:
                    off_team = team1
                else:
                    # if previous offensive team is assigned incorrectly, base it on the next offensive team
                    first_next_possession_index = end_of_possession_list[i] + 1
                    last_next_possession_index = end_of_possession_list[i+1] + 1;
                    next_possession = game_period[first_next_possession_index:last_next_possession_index]
                    next_off_team = next_possession.Team1_id.iat[-1]
                    if next_off_team == team1:
                        off_team = team2
                    elif next_off_team == team2:
                        off_team = team1
            
            # allocate possessions to players in game
            players['off_poss'], players['def_poss'] = zip(*players.apply(allocate_possessions, t=off_team, t1=team1, t2=team2, axis=1))
           
            # change players in game if there is a substitution
            possession[possession.Event_Msg_Type == 8].apply(substitute, axis=1)
        
            # previous offensive team
            previous_off_team = off_team
            
    # calculate offrtg and defrtg for this game
    players['off_rtg'] = players['off_pts'] / players['off_poss'] *100
    players['def_rtg'] = players['def_pts'] / players['def_poss'] * 100
    
    # append to list of game results
    players_list.append(players[['Game_id', 'Person_id', 'off_rtg', 'def_rtg']])


In [12]:
# concatenate all the game results
answer = pd.concat(players_list)

# rename columns
answer.columns = ['Game_ID', 'Player_ID', 'OffRtg', 'DefRtg']

answer

,Game_ID,Player_ID,OffRtg,DefRtg
0,006728e4c10e957011e1f24878e6054a,ae53f8ba6761b64a174051da817785bc,66.666667,100.000000
23,006728e4c10e957011e1f24878e6054a,e59b921ab3da55f632bc748beb12805a,NaN,NaN
21,006728e4c10e957011e1f24878e6054a,618f6d58ab2881152607c2a6e057bc51,120.408163,119.607843
19,006728e4c10e957011e1f24878e6054a,dd1da128c27db468d95b99b583f8a57d,NaN,NaN
17,006728e4c10e957011e1f24878e6054a,c10b49616a2f4a23607dc1a8be4fde9f,NaN,NaN
15,006728e4c10e957011e1f24878e6054a,f4a5ca938177c407a9dab5412e39498f,87.878788,103.225806
13,006728e4c10e957011e1f24878e6054a,c5dd5b2e3b975f0849d9b74e74125cb9,98.823529,113.095238
12,006728e4c10e957011e1f24878e6054a,42e0d7167f04a4ff958c6442da0e6851,105.633803,108.333333
10,006728e4c10e957011e1f24878e6054a,766802a8fda500d7945950de7398c9c6,101.282051,107.894737
14,006728e4c10e957011e1f24878e6054a,5db9c1c8184510fee8161e7fafdc9c49,71.794872,88.571429


In [13]:
answer.to_csv('Francis_Lin_Q1_BBALL.csv', index=False)